In [141]:
import numpy as np
import matplotlib.pyplot as plt
import math
from scipy.special import expit
import time

# Problem 1
## Dataset Generation

Write a function to **generate a training set** of size $m$
- randomly generate a weight vector $w \in \mathbb{R}^{10}$, normalize length
- generate a training set $\{(x_i , y_i)\}$ of size m
  - $x_i$: random vector in $\mathbb{R}^{10}$ from $\textbf{N}(0, I)$
  - $y_i$: $\{0, +1\}$ with $P[y = +1] = \sigma(w \cdot x_i)$ and $P[y = 0] = 1 - \sigma(w \cdot x_i)$

In [142]:
def generate_vector(normalize=False):
    v = np.random.randn(10)
    if normalize:
        sum = 0
        for i in v:
            sum += i * i 
        norm = math.sqrt(sum)
        v = [i*1/norm for i in v]
    return v

def generate_label(x_i,w):
    s = expit(np.dot(w,x_i))
    X = np.random.uniform(low=0,high=1)
    y_i = 0
    if X <= s:
        y_i = 1
    return y_i

def generate_data(m):
    x = []
    for i in range(m):
        x.append(generate_vector(normalize=False))
    x = np.array(x)
    return x

def new_data(m):
    w = generate_vector(normalize=True) 
    x = generate_data(m)
    y =[generate_label(i,w) for i in x ]
    return w, x, y 

# w_ = generate_vector(normalize=True) 
# x_m = generate_data(10)
# y_m = np.array([generate_label(i,w_) for i in x_m ])


## Algorithm 1: logistic regression

The goal is to learn $w$.  Algorithm 1 is logistic
  regression (you may use the built-in method LogisticRegression for this. Use max_iter=1000).

In [143]:
from sklearn.linear_model import LogisticRegression

def algorithm1(X,Y):
    start = time.time()
    clf = LogisticRegression(random_state=0, max_iter=1000).fit(X, Y)
    w_prime = np.array(clf.coef_[0])
    print(time.time()-start)
    return w_prime

## Algorithm 2: gradient descent with square loss

Define square loss as
$$L_i(w^{(t)}) = \frac{1}{2} \left( \sigma(w^{(t)} \cdot x) - y_i \right)^2$$

  Algorithm 2 is
  gradient descent with respect to square loss (code this
  up yourself -- run for 1000 iterations, use step size eta = 0.01).

In [148]:
def loss_gradient(x,y,w):
    sig = expit(np.dot(w,x))
    return (sig - y)*sig*(1-sig)*x

def gradient(x,y,w):
    values = []
    for i in range(len(x)):
        values.append(loss_gradient(x[i],y[i],w))
    return sum(values)

def algorithm2(x,y):
    start = time.time()
    w_t = np.zeros(10)
    for iterations in range(1000):
        w_t -= 0.01*gradient(x,y,w_t)
    print(time.time()-start)
    return w_t

## Algorithm 3: stochastic gradient descent with square loss
Similar to gradient descent, except we use the gradient at a single random training point every iteration.

In [149]:
def stocastic_gradient(x,y,w):
    random = np.random.randint(0,len(x))
    return loss_gradient(x[random],y[random],w)

def algorithm3(x,y):
    start = time.time()
    w_t = np.zeros(10)
    for iterations in range(1000):
        w_t -= 0.01*stocastic_gradient(x,y,w_t)
    print(time.time()-start)
    return w_t

## Evaluation

Measure error $\|w - \hat{w}\|_2$ for each method at different sample size. For any
  fixed value of $m$, choose many different $w$'s and average the
  values $\|w - 
  \hat{w}\|_2$ for Algorithms 1, 2 and 3.  Plot the results
  for for each algorithm as you make $m$ large (use $m=50, 100, 150, 200, 250$).
  Also record, for each algorithm, the time taken to run the overall experiment.

In [150]:

a1 = []
a2 = []
a3 = []
M = [50,100,150,200,250]

# 10 repetitions of each algorithm 
for i in range(10):
    for m in M:
        w_star, x , y = new_data(m)
        
        w_new = algorithm1(x,y)
        w_sub = w_star-w_new
        w_norm_diff = np.linalg.norm(w_sub)
        a1.append(w_norm_diff)

        w_new_2 = algorithm2(x,y)
        w_sub_2 = np.subtract(w_star,w_new_2)
        w_norm_diff_2 = np.linalg.norm(w_sub_2)
        a2.append(w_norm_diff_2)

        w_new_3 = algorithm3(x,y) 
        w_sub_3 = np.subtract(w_star,w_new_3)
        w_norm_diff_3 = np.linalg.norm(w_sub_3)
        a3.append(w_norm_diff_3)

print(np.average(a1))
print(np.average(a2))
print(np.average(a3))


0.6758945486549706
0.9712871550758766
0.6843225233208823


# Problem 2

In [126]:
from sklearn import datasets

In [127]:
cancer = datasets.load_breast_cancer()

For each depth in $1, \dots, 5$, instantiate an AdaBoost classifier with the base learner set to be a decision tree of that depth (set `n_estimators=10` and `learning_rate=1`), and then record the 10-fold cross-validated error on the entire breast cancer data set. Plot the resulting curve of accuracy against base classifier depth. Use $101$ as your random state for both the base learner as well as the AdaBoost classifier every time.